# Notebook: Einführung inPrompting-Strategien für Large Language Models (LLMs)

In diesem Notebook werden verschiedene Prompting-Strategien vorgestellt, die helfen können, die Leistung von Large Language Models (LLMs) zu verbessern.

## 📚 Quellen

- [Prompting Guide](https://www.promptingguide.ai/de/techniques)
- [Python-Paket `openai`](https://github.com/openai/openai-python)

---

Viel Spaß beim Erkunden und Experimentieren! 🤗

Wir verwenden das `openai`-Paket, um mit einem LLM zu interagieren. Das Paket erlaubt sowohl die Kommunikation mit der OpenAI-API als auch mit anderen LLMs, die eine kompatible API anbieten. 

**Wichtiger Hinweis:** Bei der Uni haben wir mehrere Large Language Models zur Verfügung gestellt über einen Server. Ich verweise an dieser Stelle auf die Anleitung für den VPN-Zugang, den ich bei GRIPS hinterlegt habe. Anfragen an das LLM sind nur **innerhalb des Uni-Netzwerks** möglich, das bedeutet, dass ihr entweder auf dem Campus seid im `eduroam` oder über VPN verbunden seid.

Auf dem Uni-Server werden mehrere Open-Source LLMs angeboten, wobei diese ollama bereitgestellt werden.

Wir werden nun das Packet installieren.

In [1]:
%%capture 
!pip install openai

In [2]:
# Importieren wir nun das Paket
from openai import OpenAI

In [3]:
# Definieren wir die API-URL und den API-Schlüssel. Beachte den Hinweis oben!
LLM_URL = "http://132.199.138.16:11434/v1"
LLM_MODEL = "gemma3:4b"

In [4]:
client = OpenAI(
    base_url=LLM_URL,
    api_key='ollama',
)

In [5]:
# Definieren wir eine Funktion, die eine Anfrage an das LLM stellt. Dabei handelt es sich um die completions-API, die eine Next-Token-Vorhersage durchführt.
# Es werden maximal `max_tokens` generiert, die durch das `stop`-Token begrenzt werden können.
def llm_completion(prompt, temperature=0.0, stop=["\n"], seed=0, max_tokens=256):
    completion = client.completions.create(
        model=LLM_MODEL,
        temperature=temperature,
        stop=stop,
        prompt=prompt,
        max_tokens=max_tokens,
        seed=seed
    )
    return completion.choices[0].text

In [6]:
# Definieren wir noch eine weitere Funktion, die die Chat-Completions-API nutzt. Diese erlaubt es, Nachrichten in einem Chat-Format zu übergeben.
def llm_completion_chat(prompt, temperature=0.0, stop=None, image_url=None, seed=0):
    messages = [
        {
            "role": "user",
            "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
            ] + ([{"type": "image_url", "image_url": {"url": image_url}}] if image_url else [])
        }
    ]

    extra_body = {"stop": [stop]} if stop else {}

    response = client.chat.completions.create(
        model=LLM_MODEL,
        messages=messages,
        temperature=temperature,
        stream=False,
        seed=seed,
        extra_body=extra_body,
    )

    return response.choices[0].message.content

## 1. Zero-shot Prompting

**Zero-shot Prompting** bedeutet, dass Sie dem LLM direkt eine Aufgabe stellen – ganz ohne Beispiele oder zusätzliche Erklärungen.  

Das Modell nutzt sein trainiertes Wissen, um Ihre Anfrage bestmöglich zu beantworten. Oft funktioniert das überraschend gut, besonders bei klar formulierten Fragen.

In [7]:
prompt = "Paris ist die Hauptstadt von?"
print(llm_completion_chat(prompt, stop="\n"))

Paris ist die Hauptstadt von Frankreich.


## 2. Few-shot Prompting

**Few-shot Prompting** bedeutet, dass Sie dem LLM einige Beispiele geben, um mehr Kontext zu bieten.

Dies hilft dem Modell, Ihre Anfrage besser zu verstehen und relevantere Antworten zu generieren.

In dem Kontext ist das Paper **„Language Models are Few-Shot Learners“** von Brown et al. (2020), das 2020 GPT-3 vorgestellt hatte von besonderer Bedeutung. Statt das Modell neu zu trainieren, reicht es bei LLMs, dem Modell wenige Beispiele direkt im Prompt zur Verfügung zu stellen (Few-Shot), um neue Aufgaben zu lösen. Es kann in gewisser Weise von einem Paradigmenwechsel gesprochen werden: Statt für jede Aufgabe ein Modell feinzujustieren, können große Modelle durch Prompt-Engineering generalisieren.

[Hier geht es zum Paper](https://proceedings.neurips.cc/paper_files/paper/2020/file/1457c0d6bfcb4967418bfb8ac142f64a-Paper.pdf)

In [17]:
# In dem folgenden Beispiel wird ein Prompt verwendet, um Sentiment-Elemente aus Sätzen zu extrahieren.
# Das Modell soll die relevanten Elemente identifizieren und deren Sentiment klassifizieren.
prompt = '''Satz: Es war richtig toll in Berlin.
Sentiment Elemente: [("Berlin", "positive")]
Satz: Das Essem war nicht so lecker.
Sentiment Elemente: [("Essen", "negative")]
Satz: Hier in Portugal gibt es fantastische Strände aber leider war das Wetter schlecht.
Sentiment Elemente: [("Strände", "positive"), ("Wetter", "negative")]
Satz: Die Stadt ist sehr schön.
Sentiment Elemente: '''

# stop ist ein optionales Stop-Kriterium, um die Antwort zu beenden
print(llm_completion(prompt, stop="\n"))  # Stoppt die Antwort bei "Satz:"

Sentiment Elemente: [("Stadt", "positive")]


### 3. Chain of Thought

Eingeführt in [Wei et al. (2022)](https://arxiv.org/abs/2201.11903), ermöglicht Chain-of-Thought (CoT) Prompting komplexe Schlussfolgerungsfähigkeiten durch Zwischenschritte im Denkprozess. Sie können es mit Few-Shot-Prompting kombinieren, um bessere Ergebnisse bei komplexeren Aufgaben zu erzielen, die eine Schlussfolgerung vor der Beantwortung erfordern.

In [9]:
prompt = '''Ich ging auf den Markt und kaufte 10 Äpfel. Ich gab 2 Äpfel an den Nachbarn und 2 an den Handwerker. Dann ging ich und kaufte 5 weitere Äpfel und aß 1. Wie viele Äpfel blieben mir übrig?
Lass uns Schritt für Schritt denken.\n'''

print(llm_completion_chat(prompt))  # Stoppt die Antwort bei "Lass uns Schritt

Okay, lass uns das Schritt für Schritt durchgehen:

1. **Start:** Du hast mit 10 Äpfeln angefangen.
2. **An den Nachbarn:** Du hast 2 Äpfel gegeben, also bleiben 10 - 2 = 8 Äpfel.
3. **An den Handwerker:** Du hast 2 Äpfel gegeben, also bleiben 8 - 2 = 6 Äpfel.
4. **Neue Käufe:** Du kaufst 5 Äpfel dazu, also hast du 6 + 5 = 11 Äpfel.
5. **Du isst einen Apfel:** Du isst 1 Apfel, also bleiben 11 - 1 = 10 Äpfel.

**Antwort:** Du hast 10 Äpfel übrig.


## 4. Self-Consistency / Majority Vote

Self-Consistency ist eine Technik, die in [Wang et al. (2022)](https://arxiv.org/abs/2203.11171) eingeführt wurde. Sie nutzt die Tatsache, dass LLMs bei der Beantwortung von Fragen oft mehrere plausible Antworten generieren können. Statt sich auf eine einzelne Antwort zu verlassen, aggregiert Self-Consistency mehrere Antworten und wählt die häufigste Antwort aus, um die Genauigkeit zu erhöhen.

In dem Beispiel unten generieren wir mehrere Antworten, indem wir die `temperature` auf 0.8 setzen, und wählen dann die häufigste Antwort aus. Ich erinnere an die Vorlesung, wobei wir die `temperature` auf 0 gesetzt hatten, um eine deterministische Antwort zu erhalten, 0.8 sorgt für mehr Vielfalt in den Antworten, da nicht das Token mit der höchsten Wahrscheinlichkeit immer ausgewählt wird.

In [ ]:
import re

prompt = '''Q: Anna hat 2 Äpfel. Sie bekommt 3 weitere Äpfel von ihrer Freundin. Wie viele Äpfel hat sie jetzt?
A: Anna hat 2 Äpfel. Sie bekommt 3 weitere. 2 + 3 = 5. Die Antwort ist 5.

Q: Tom hat 7 Buntstifte. Er kauft 5 weitere. Wie viele Buntstifte hat er insgesamt?
A: Tom hat 7 Buntstifte. Er kauft 5 weitere. 7 + 5 = 12. Die Antwort ist 12.

Q: Lisa hat 10 Euro. Sie bekommt 4 Euro Taschengeld. Wie viel Geld hat sie danach?
A: Lisa hat 10 Euro. Sie bekommt 4 Euro dazu. 10 + 4 = 14. Die Antwort ist 14.

Q: Paul hat 3 Tafeln Schokolade. Er bekommt von seinem Bruder 6 Tafeln Schokolade dazu. Wie viel Schokolade hat Paul?
A: '''

# Wir können nun Self-Consistency aufrufen
predictions = [llm_completion(prompt, stop="\n\n", seed=i, temperature=0.8) for i in range(5)]

prediction_ints = []
for p in predictions:
    numbers = re.findall(r'\d+', p)
    if numbers:
        # Wir nehmen die letzte gefundene Zahl als Vorhersage
        prediction_ints.append(int(numbers[-1]))
    else:
        prediction_ints.append(None)

# Ausgabe: fünf Vorhersagen + häufigste Antwort
prediction_ints, max(set(prediction_ints), key=prediction_ints.count)


([9, 9, 9, 9, 9], 9)

## 5. Generiertes Wissens-Prompting

Große Sprachmodelle (LLMs) werden kontinuierlich verbessert, und eine beliebte Technik beinhaltet die Fähigkeit, Wissen oder Informationen einzubinden, um dem Modell zu helfen, genauere Vorhersagen zu treffen.

Kann das Modell mit einer ähnlichen Idee auch genutzt werden, um Wissen zu generieren, bevor eine Vorhersage getroffen wird? Genau das wird im Paper von [Liu et al. 2022](https://arxiv.org/pdf/2110.08387.pdf) versucht – Wissen zu generieren, das als Teil des Prompts verwendet wird. Wie nützlich ist dies für Aufgaben wie Schlussfolgerungen nach gesundem Menschverstand?

Schritte:

1. Zuerst generieren wir einige "Wissensstände":
2. Dann verwenden wir diese Wissensstände, um den Prompt zu erweitern.
3. Schließlich verwenden wir den erweiterten Prompt, um die Vorhersage zu treffen.

In [16]:
prompt = '''Eingabe: Griechenland ist größer als Mexiko.
Wissen: Griechenland ist ungefähr 131.957 Quadratkilometer groß, während Mexiko ungefähr 1.964.375 Quadratkilometer groß ist. Mexiko ist daher 1.389% größer als Griechenland.

Eingabe: Brillen beschlagen immer.
Wissen: Kondensation tritt auf Brillengläsern auf, wenn Wasserdampf aus Ihrem Schweiß, Atem und der umgebenden Feuchtigkeit auf eine kalte Oberfläche trifft, abkühlt und sich dann in winzige Flüssigkeitströpfchen verwandelt und einen Film bildet, den Sie als Beschlag wahrnehmen. Ihre Gläser werden im Vergleich zu Ihrem Atem relativ kühl sein, besonders wenn die Außenluft kalt ist.

Eingabe: Ein Fisch ist in der Lage zu denken.
Wissen: Fische sind intelligenter, als sie scheinen. In vielen Bereichen, wie beispielsweise dem Gedächtnis, stehen ihre kognitiven Fähigkeiten denen von 'höheren' Wirbeltieren, einschließlich nichtmenschlicher Primaten, in nichts nach. Die Langzeitgedächtnisse der Fische helfen ihnen, komplexe soziale Beziehungen im Überblick zu behalten.

Eingabe: Eine häufige Wirkung des Rauchens vieler Zigaretten im Laufe des Lebens ist eine überdurchschnittlich hohe Wahrscheinlichkeit, Lungenkrebs zu bekommen.
Wissen: Diejenigen, die konstant weniger als eine Zigarette pro Tag im Laufe ihres Lebens geraucht haben, hatten ein neunmal höheres Risiko an Lungenkrebs zu sterben als Nichtraucher. Bei Personen, die zwischen einer und 10 Zigaretten pro Tag rauchten, war das Risiko an Lungenkrebs zu sterben fast 12 Mal höher als bei Nichtrauchern.

Eingabe: Wie viele Planeten gibt es im Sonnensystem?
Wissen: '''

wissen = llm_completion(prompt, stop="\n")

prompt_mit_wissen = f'''Frage: Ist die Erde ist der einzige Planet im Sonnensystem?"
Wissen: {wissen}
Antwort:'''

antwort = llm_completion(prompt_mit_wissen, stop="\n")
print(prompt_mit_wissen + antwort)

Frage: Ist die Erde ist der einzige Planet im Sonnensystem?"
Wissen: Wissen: Es gibt acht Planeten in unserem Sonnensystem: Merkur, Venus, Erde, Mars, Jupiter, Saturn, Uranus und Neptun.
Antwort:Nein, die Erde ist nicht der einzige Planet im Sonnensystem. Es gibt acht Planeten: Merkur, Venus, Erde, Mars, Jupiter, Saturn, Uranus und Neptun.


### Aufgabe 1: Named Entity Recognition mit Few-Shot Learning

#### Ziel der Aufgabe
Entwickeln Sie eine Few-Shot-Prompt-Strategie zur automatischen Erkennung von **Ortsnamen** (LOC - Location Entities) in deutschen Texten.  
Als Datengrundlage dient die vorbereitete Datei **`LOC_sentences.txt`**, die Sätze im Format  

```
Satz####["Entity1", "Entity2", ...]
```

enthält. Jeder Eintrag besteht also aus einem deutschen Satz und der zugehörigen Liste aller Ortsnamen (LOC-Entities) in diesem Satz.  

#### Anforderungen
1. **Few-Shot Learning:** Erstellen Sie eine Prompt mit **fünf zufälligen Few-shot Beispielen** aus `LOC_sentences.txt`, die das gewünschte Input-Output-Verhalten demonstrieren.  
2. **Ausgabeformat:** Das Modell soll erkannte Ortsnamen als String-Liste zurückgeben (z. B. `["Berlin", "München"]`).  
3. **Evaluation:** Testen Sie Ihr Few-Shot-Prompting auf 64 zufälligen Beispielen aus `LOC_sentences.txt`. 

#### Evaluation mit Accuracy
Zur Bewertung der Ergebnisse soll die **Accuracy** berechnet werden.  

Ein Satz gilt als **korrekt gelabelt**, wenn die vorhergesagte Liste von Ortsnamen exakt mit der Gold-Standard-Liste aus `LOC_sentences.txt` übereinstimmt.

#### Beispiel für das erwartete Verhalten

```md
Input: "In Essen haben sich die beiden Brüder getroffen."
Output: ["Essen"]
```

Tipp: Mit `eval()` können Sie die String-Ausgabe des Modells in eine Python-Liste umwandeln.

#### Beispiel Prompt

```text
Erkenne alle Ortsnamen in deutschen Sätzen und gib sie als Liste zurück.

Satz: Die neuerliche Verwaltungsreform aus dem Jahre 1815 brachte für Passenheim abermals eine neue Kreiszuordnung mit sich .
Ortsnamen: ['Passenheim']

Satz: Die Hauptbasis wurde 2006 von Alderney nach Jersey verlegt .
Ortsnamen: ['Alderney', 'Jersey']

Satz: Sie verläuft zunächst entlang der Bayernstraße , überquert diese und erreicht die Haltestelle Dutzendteich , an der Umsteigemöglichkeit zur S-Bahn nach Altdorf besteht .
Ortsnamen: ['Bayernstraße', 'Dutzendteich', 'Altdorf']

Satz: Rhein-Kreis Neuss Eigentlich kommt das Thema immer zum Jahresanfang auf den Tisch , wenn die Fraktionen über den Haushaltsplan beraten : Soll sich der Rhein-Kreis Neuss von seinen RWE-Aktien trennen ?
Ortsnamen: ['Rhein-Kreis Neuss', 'Rhein-Kreis Neuss']

Satz: Eine Goevier ist in der Flugwerft Schleißheim ausgestellt .
Ortsnamen: ['Schleißheim']

Satz: Heute ist durch einen roten Ziegelstreifen der Verlauf des an den Turm anschließenden nordwestlichen Mauerrings gekennzeichnet , während der südwestliche Mauerring bis zum Gelände des ehemaligen Franziskanerklosters St . Johannis noch intakt ist .
Ortsnamen:
```

In [12]:
# Laden des Datensatzes
dataset = []

with open("LOC_sentences.txt", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        sentence, entities_str = line.split("####")
        entities = eval(entities_str)
        dataset += [[sentence, entities]]

print("Satz:", dataset[0][0])
print("Entities:", dataset[0][1])

# 5 zufällige Few-Shot Beispiele
import random
random.seed(42)
few_shot_examples = random.sample(dataset, 5)
test_data = [item for item in dataset if item not in few_shot_examples][:64]

Satz: Heute ist durch einen roten Ziegelstreifen der Verlauf des an den Turm anschließenden nordwestlichen Mauerrings gekennzeichnet , während der südwestliche Mauerring bis zum Gelände des ehemaligen Franziskanerklosters St . Johannis noch intakt ist .
Entities: ['St . Johannis']


In [13]:
# Ihr Code steht hier ...

<details>
<summary><b>Lösung anzeigen</b></summary>

```python
# Prompt erstellen
def create_prompt(examples, test_sentence):
    prompt = "Erkenne alle Ortsnamen in deutschen Sätzen und gib sie als Liste zurück.\n\n"
    
    for sentence, entities in examples:
        prompt += f"Satz: {sentence}\nOrtsnamen: {entities}\n\n"
    
    prompt += f"Satz: {test_sentence}\nOrtsnamen:"
    return prompt

# Evaluation
def evaluate(test_data):
    correct = 0
    total = len(test_data)
    
    for sentence, true_entities in test_data:
        prompt = create_prompt(few_shot_examples, sentence)
        response = llm_completion(prompt, stop="\n")
        
        # Da das LLM manchmal ungültige Python-Ausdrücke zurückgibt, verwenden wir try-except
        try:
            predicted_entities = eval(response.strip())
            if set(predicted_entities) == set(true_entities): # Mit set vergleichen, um Reihenfolge zu ignorieren
                correct += 1
        except:
            pass
    
    accuracy = correct / total
    return accuracy

# Ausführung
accuracy = evaluate(test_data) * 100
print(f"Accuracy: {accuracy:.2f}")
```

</details>